<a href="https://colab.research.google.com/github/NathanielCogneaux/NLP-comparison-between-Naive-Bayes-RNN-LSTM-and-Transformers/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>